## 배열 연산 : 브로드캐스팅(Broadcasting)
<img src="http://www.astroml.org/_images/fig_broadcast_visual_1.png">
브로드캐스팅(Broadcasting)은 모양이 다른 배열들 간의 연산이 어떤 조건을 만족했을 때 가능해지도록 배열을 자동적으로 변환하는 것이라고 정의할 수 있다. 그리고 누락되었거나 길이가 1인 차원에 대해 브로드캐스팅이 수행된다.

위의 세 그림은 배열 연산에서 항상 1차원 배열이 포함되어 있다. 그리고 행/열의 어떤 축이든(;0번/1번 어떤 축이든) 차원의 길이가 같은 것을 알 수 있다. 3x3 + 1x3,  3x1 + 1x3

브로드캐스팅이 일어날 수 있는 조건은 다음과 같다.
차원의 크기가 1일때 가능하다
두 배열 간의 연산에서 최소한 하나의 배열의 차원이 1이라면(0번 축이든 1번 축이든; 1행이든 1열이든) 가능하다.
차원의 짝이 맞을 때 가능하다
차원에 대해 축의 길이가 동일하면 브로드캐스팅이 가능하다.


### 브로드캐스팅을 이용하여 국소별 거리 구하기

for문에 안좋은 추억이 있는 나를 위해 
브로드캐스팅을 이용하여 거리를 구해보자!! 

In [103]:
import numpy as np
import pandas as pd
import time
startTime = time.time()

In [104]:
# 102국소
sam_dis=pd.read_excel('gn\sample.xlsx')

In [105]:
sam_dis.head(2)

,Unnamed: 0,enb_id,cellnum,enb_cell,pci,frequency,enb_pci,etl_dt,freq_classify,bts_name,...,cell_lat,cell_lng,mcp_nm,sgg_nm,emd_nm,freq_code,part_id,part_name,freq_cnt,sigu_nm
0,0,34288,2,34288_2,399,4.0,34288_399,20190224,2.6GHz(10M),SKB해운대국사_해운대좌동L0,...,35.168056,129.179167,부산,해운대구,좌동,4,1101,경남/동부산품질개선팀,4,부산해운대구
1,1,34288,1,34288_1,399,9.0,34288_399,20190224,2.6GHz(20M),SKB해운대국사_해운대좌동L0,...,35.200833,129.152500,부산,해운대구,반여동,9,1101,경남/동부산품질개선팀,4,부산해운대구


In [106]:
# 이름없는 Column 이름한번 지어주고
sam_dis.columns.values[0]='id1'
sam_dis.inplace=True

In [107]:
# 그냥 인덱스도 Reset 해주고
sam_dis.reset_index(inplace=True)

In [108]:
sam_dis=sam_dis[['id1','enb_cell','cell_nm','cell_lat','cell_lng']]

In [109]:
# 위경도를 Array 
t_lat=np.array(sam_dis['cell_lat'])
t_lon=np.array(sam_dis['cell_lng'])

In [110]:
sam_dis.head(3)

,id1,enb_cell,cell_nm,cell_lat,cell_lng
0,0,34288_2,haeundaejwadongL0_2,35.168056,129.179167
1,1,34288_1,해운대장산2LRRU_F5,35.200833,129.152500
2,2,34288_0,해운대장산2LRRUM,35.198923,129.150121


In [111]:
len(t_lat)

102

In [112]:
# 행으로 세로모양 Array
t_lat1=t_lat.reshape(len(t_lat),1)

In [113]:
# 열로 가로모양 Array
t_lat2=t_lat.reshape(1,len(t_lat))
# t_lat1.T 와 같다

In [114]:
# 경도 도 위도와 같이 변경
t_lon1=t_lon.reshape(len(t_lon),1)
t_lon2=t_lon.reshape(1,len(t_lon))

In [115]:
# 위경도 거리구하는 공식
d_lat=np.radians(t_lat2)-np.radians(t_lat1)

In [116]:
d_lon=np.radians(t_lon2)-np.radians(t_lon1)

In [117]:
a=np.sin(d_lat/2)**2+np.cos(t_lat1)*np.cos(t_lat2)*np.sin(d_lon/2)**2

In [118]:
c=2*np.arcsin(np.sqrt(a))

In [119]:
c

array([[0.00000000e+00, 6.85040717e-04, 6.77460182e-04, ...,
        1.06770427e-03, 1.06770427e-03, 1.06444566e-03],
       [6.85040717e-04, 0.00000000e+00, 4.70882893e-05, ...,
        4.70833731e-04, 4.70833731e-04, 4.66026420e-04],
       [6.77460182e-04, 4.70882893e-05, 0.00000000e+00, ...,
        5.11005421e-04, 5.11005421e-04, 5.06093584e-04],
       ...,
       [1.06770427e-03, 4.70833731e-04, 5.11005421e-04, ...,
        0.00000000e+00, 0.00000000e+00, 5.28361524e-06],
       [1.06770427e-03, 4.70833731e-04, 5.11005421e-04, ...,
        0.00000000e+00, 0.00000000e+00, 5.28361524e-06],
       [1.06444566e-03, 4.66026420e-04, 5.06093584e-04, ...,
        5.28361524e-06, 5.28361524e-06, 0.00000000e+00]])

In [120]:
# km로 변경
km=6371*c
km

array([[0.        , 4.36439441, 4.31609882, ..., 6.80234392, 6.80234392,
        6.78158329],
       [4.36439441, 0.        , 0.29999949, ..., 2.9996817 , 2.9996817 ,
        2.96905432],
       [4.31609882, 0.29999949, 0.        , ..., 3.25561554, 3.25561554,
        3.22432222],
       ...,
       [6.80234392, 2.9996817 , 3.25561554, ..., 0.        , 0.        ,
        0.03366191],
       [6.80234392, 2.9996817 , 3.25561554, ..., 0.        , 0.        ,
        0.03366191],
       [6.78158329, 2.96905432, 3.22432222, ..., 0.03366191, 0.03366191,
        0.        ]])

In [121]:
# 결과를 DataFrame에 넣어보자
dis_pd=pd.DataFrame(km)
dis_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,0.000000,4.364394,4.316099,4.316099,0.000000,4.451568,4.515051,3.853164,1.991469,2.813556,...,38.535655,36.887877,36.887877,37.663424,37.663424,6.276808,6.276808,6.802344,6.802344,6.781583
1,4.364394,0.000000,0.299999,0.299999,4.364394,1.068839,0.875468,1.336557,2.374271,1.607304,...,38.129412,36.010161,36.010161,36.667855,36.667855,2.162130,2.162130,2.999682,2.999682,2.969054
2,4.316099,0.299999,0.000000,0.000000,4.316099,1.362342,1.173836,1.572100,2.327956,1.639873,...,37.838735,35.727954,35.727954,36.388560,36.388560,2.383858,2.383858,3.255616,3.255616,3.224322
3,4.316099,0.299999,0.000000,0.000000,4.316099,1.362342,1.173836,1.572100,2.327956,1.639873,...,37.838735,35.727954,35.727954,36.388560,36.388560,2.383858,2.383858,3.255616,3.255616,3.224322
4,0.000000,4.364394,4.316099,4.316099,0.000000,4.451568,4.515051,3.853164,1.991469,2.813556,...,38.535655,36.887877,36.887877,37.663424,37.663424,6.276808,6.276808,6.802344,6.802344,6.781583


In [122]:
# 행열 이름도 한꺼번에 넣을때
cellnm=sam_dis['cell_nm']
dis_pd2=pd.DataFrame(km,index=[cellnm],columns=[cellnm])
dis_pd2.head(5)

cell_nm,haeundaejwadongL0_2,해운대장산2LRRU_F5,해운대장산2LRRUM,해운대장산2LRRU,haeundaejwadongL0_6,해운대장산4LRRU_F5,해운대장산4LRRU,해운대장산4LRRUM,폭포사LRRUM,폭포사LRRU,...,강서가덕도등대LRRU_F1,거가대로가덕터널2LRRU,거가대로가덕터널2LRRUM,강서천성두문LRRU,강서천성두문LRRUM,(CRM)해운대반송동6LRRU,(CRM)해운대반송동6LRRUM,반송벽산한솔아파트LRRU,반송벽산한솔아파트LRRUM,반송벽산한솔아파트LRRU_F3
cell_nm,,,,,,,,,,,,,,,,,,,,,
haeundaejwadongL0_2,0.000000,4.364394,4.316099,4.316099,0.000000,4.451568,4.515051,3.853164,1.991469,2.813556,...,38.535655,36.887877,36.887877,37.663424,37.663424,6.276808,6.276808,6.802344,6.802344,6.781583
해운대장산2LRRU_F5,4.364394,0.000000,0.299999,0.299999,4.364394,1.068839,0.875468,1.336557,2.374271,1.607304,...,38.129412,36.010161,36.010161,36.667855,36.667855,2.162130,2.162130,2.999682,2.999682,2.969054
해운대장산2LRRUM,4.316099,0.299999,0.000000,0.000000,4.316099,1.362342,1.173836,1.572100,2.327956,1.639873,...,37.838735,35.727954,35.727954,36.388560,36.388560,2.383858,2.383858,3.255616,3.255616,3.224322
해운대장산2LRRU,4.316099,0.299999,0.000000,0.000000,4.316099,1.362342,1.173836,1.572100,2.327956,1.639873,...,37.838735,35.727954,35.727954,36.388560,36.388560,2.383858,2.383858,3.255616,3.255616,3.224322
haeundaejwadongL0_6,0.000000,4.364394,4.316099,4.316099,0.000000,4.451568,4.515051,3.853164,1.991469,2.813556,...,38.535655,36.887877,36.887877,37.663424,37.663424,6.276808,6.276808,6.802344,6.802344,6.781583


In [123]:
# merge를 이용해서
dis_pd.columns=[sam_dis['cell_nm']]
cell_nm=sam_dis[['cell_nm']]
dis_pd_merge=pd.merge(cell_nm,dis_pd,left_index=True,right_index=True)
dis_pd_merge.head()

,cell_nm,"(haeundaejwadongL0_2,)","(해운대장산2LRRU_F5,)","(해운대장산2LRRUM,)","(해운대장산2LRRU,)","(haeundaejwadongL0_6,)","(해운대장산4LRRU_F5,)","(해운대장산4LRRU,)","(해운대장산4LRRUM,)","(폭포사LRRUM,)",...,"(강서가덕도등대LRRU_F1,)","(거가대로가덕터널2LRRU,)","(거가대로가덕터널2LRRUM,)","(강서천성두문LRRU,)","(강서천성두문LRRUM,)","((CRM)해운대반송동6LRRU,)","((CRM)해운대반송동6LRRUM,)","(반송벽산한솔아파트LRRU,)","(반송벽산한솔아파트LRRUM,)","(반송벽산한솔아파트LRRU_F3,)"
0,haeundaejwadongL0_2,0.000000,4.364394,4.316099,4.316099,0.000000,4.451568,4.515051,3.853164,1.991469,...,38.535655,36.887877,36.887877,37.663424,37.663424,6.276808,6.276808,6.802344,6.802344,6.781583
1,해운대장산2LRRU_F5,4.364394,0.000000,0.299999,0.299999,4.364394,1.068839,0.875468,1.336557,2.374271,...,38.129412,36.010161,36.010161,36.667855,36.667855,2.162130,2.162130,2.999682,2.999682,2.969054
2,해운대장산2LRRUM,4.316099,0.299999,0.000000,0.000000,4.316099,1.362342,1.173836,1.572100,2.327956,...,37.838735,35.727954,35.727954,36.388560,36.388560,2.383858,2.383858,3.255616,3.255616,3.224322
3,해운대장산2LRRU,4.316099,0.299999,0.000000,0.000000,4.316099,1.362342,1.173836,1.572100,2.327956,...,37.838735,35.727954,35.727954,36.388560,36.388560,2.383858,2.383858,3.255616,3.255616,3.224322
4,haeundaejwadongL0_6,0.000000,4.364394,4.316099,4.316099,0.000000,4.451568,4.515051,3.853164,1.991469,...,38.535655,36.887877,36.887877,37.663424,37.663424,6.276808,6.276808,6.802344,6.802344,6.781583


## 국소별 거리가 1km 미만국소를 찾아보자

In [124]:
km

array([[0.        , 4.36439441, 4.31609882, ..., 6.80234392, 6.80234392,
        6.78158329],
       [4.36439441, 0.        , 0.29999949, ..., 2.9996817 , 2.9996817 ,
        2.96905432],
       [4.31609882, 0.29999949, 0.        , ..., 3.25561554, 3.25561554,
        3.22432222],
       ...,
       [6.80234392, 2.9996817 , 3.25561554, ..., 0.        , 0.        ,
        0.03366191],
       [6.80234392, 2.9996817 , 3.25561554, ..., 0.        , 0.        ,
        0.03366191],
       [6.78158329, 2.96905432, 3.22432222, ..., 0.03366191, 0.03366191,
        0.        ]])

In [125]:
# 거리가 0 이상 1km 미만 국소 
km2=(km<1) & (km>0)

In [126]:
km2

array([[False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False,  True],
       [False, False, False, ..., False, False,  True],
       [False, False, False, ...,  True,  True, False]])

In [127]:
# 위조건에 맞는 (True =1) 값의 인덱스를 찾는다
km2=np.argwhere(km2==1)

In [128]:
km2

array([[  1,   2],
       [  1,   3],
       [  1,   6],
       [  2,   1],
       [  3,   1],
       [  5,   6],
       [  5,   7],
       [  6,   1],
       [  6,   5],
       [  6,   7],
       [  7,   5],
       [  7,   6],
       [  8,   9],
       [  8,  11],
       [  9,   8],
       [  9,  11],
       [ 11,   8],
       [ 11,   9],
       [ 22,  24],
       [ 24,  22],
       [ 25,  26],
       [ 25,  27],
       [ 25,  28],
       [ 25,  29],
       [ 25,  31],
       [ 25,  32],
       [ 25,  33],
       [ 25,  34],
       [ 25,  35],
       [ 25,  40],
       [ 25,  41],
       [ 25,  42],
       [ 25,  43],
       [ 25,  44],
       [ 26,  25],
       [ 26,  27],
       [ 26,  28],
       [ 26,  29],
       [ 26,  30],
       [ 26,  31],
       [ 26,  32],
       [ 26,  33],
       [ 26,  34],
       [ 26,  35],
       [ 26,  41],
       [ 26,  42],
       [ 26,  43],
       [ 26,  44],
       [ 27,  25],
       [ 27,  26],
       [ 27,  28],
       [ 27,  29],
       [ 27,

In [129]:
# km2를 DataFrame으로
km_pd=pd.DataFrame(km2)

In [130]:
km_pd.head(3)

,0,1
0,1,2
1,1,3
2,1,6


In [131]:
# 항목명을 바꾼다
km_pd.rename(columns={0:'x',1:'y'},inplace=True)

In [132]:
# 국소명을 붙여본다.
result1=pd.merge(km_pd,cell_nm,left_on='x',right_index=True)
result2=pd.merge(result1,cell_nm,left_on='y',right_index=True)

In [133]:
result2.head()

,x,y,cell_nm_x,cell_nm_y
0,1,2,해운대장산2LRRU_F5,해운대장산2LRRUM
1,1,3,해운대장산2LRRU_F5,해운대장산2LRRU
2,1,6,해운대장산2LRRU_F5,해운대장산4LRRU
5,5,6,해운대장산4LRRU_F5,해운대장산4LRRU
11,7,6,해운대장산4LRRUM,해운대장산4LRRU


In [134]:
# 거리차도 붙여본다.
result2['distance']=km[result2['x'],result2['y']]

In [135]:
#완료
result2.head()

,x,y,cell_nm_x,cell_nm_y,distance
0,1,2,해운대장산2LRRU_F5,해운대장산2LRRUM,0.299999
1,1,3,해운대장산2LRRU_F5,해운대장산2LRRU,0.299999
2,1,6,해운대장산2LRRU_F5,해운대장산4LRRU,0.875468
5,5,6,해운대장산4LRRU_F5,해운대장산4LRRU,0.223012
11,7,6,해운대장산4LRRUM,해운대장산4LRRU,0.799817


In [136]:
# 코드 수행시간
endTime = time.time() - startTime
print(endTime) 

0.6969301700592041
